In [1]:
%%capture
!pip install xformers


In [6]:
from unsloth import FastLanguageModel
import torch
print(torch.cuda.is_available())  
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

True
==((====))==  Unsloth 2024.12.12: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: NVIDIA GeForce RTX 2060 SUPER. Max memory: 8.0 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.5.1. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

c:\Users\nikita\anaconda3\envs\unsloth_env\Lib\site-packages\unsloth\models\llama.py:1177: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\cb\pytorch_1000000000000\work\c10/cuda/CUDAAllocatorConfig.h:28.)
  self.register_buffer("cos_cached", emb.cos().to(dtype=dtype, device=device, non_blocking=True), persistent=False)


generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

: 

In [7]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.12.12 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


: 

In [ ]:
from datasets import load_dataset
from unsloth import to_sharegpt, standardize_sharegpt, apply_chat_template

# Load your dataset from a CSV file
dataset = load_dataset('csv', data_files='./data.csv', split="train")

# Print the column names and a sample to understand the data
print(dataset.column_names)
print(dataset[0])

# Process the dataset using to_sharegpt
dataset_simple = to_sharegpt(
    dataset,
    merged_prompt="[[This recipe is for {name}.\n]][[Ingredients include {ingredients}.\n]]. Recipe url: {url}",
    output_column_name="url",  # Assuming 'url' is the output column
)

# Create a 'conversations' column if it doesn't exist
if 'conversations' not in dataset.column_names:
    dataset = dataset.map(
        lambda x: {"conversations": [{"role": "user", "content": f"This recipe is for {x['name']}.\nIngredients include {x['ingredients']}.\nRecipe url: {x['url']}"}]},
        desc="Creating conversations column"
    )

# Standardize the dataset
dataset = standardize_sharegpt(dataset)


['url', 'name', 'ingredients']
{'url': 'https://www.povarenok.ru/recipes/show/164365/', 'name': 'Густой молочно-клубничный коктейль', 'ingredients': "{'Молоко': '250 мл', 'Клубника': '200 г', 'Сахар': '15 г'}"}


: 

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

KeyError: 'text'

: 